In [41]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
from openaq import OpenAQ
from meteostat import Hourly, Point

client = OpenAQ(api_key='e6b162759d6b804ebcb8e855fe46395c13d93c032dc5c5f22f551a530ee09cfe')

# Tọa độ của Hà Nội
latitude = 21.0285
longitude = 105.8544
radius = 5000  # 5km

# Giá trị cào
start_date = datetime(2024, 1, 1)
end_date = datetime(2025, 1, 1)

data = 'measurements'
rollup = 'hourly'

# Weather
hanoi = Point(21.0285, 105.8542)


In [42]:
# Lấy các trạm đo trong bán kính
sensor_radius_data = []
sensor_radius_list = []

country = client.locations.list(
    radius=radius,
    coordinates=(latitude, longitude)
)

if country.results:
    for location in country.results:
        for sensor in location.sensors:
            sensor_radius_list.append(sensor.id)

            sensor_radius_data.append({
                "Location Name": location.name,
                "Country": location.country.name,
                "Timezone": location.timezone,
                "Sensor ID": sensor.id,
                "Sensor Name": sensor.name,
                "Parameter": sensor.parameter.name,
                "Units": sensor.parameter.units
            })

df_sensors = pd.DataFrame(sensor_radius_data)

print(df_sensors)
print(sensor_radius_list)

                Location Name  Country      Timezone  Sensor ID Sensor Name  \
0   US Diplomatic Post: Hanoi  Vietnam  Asia/Bangkok       5049  pm25 µg/m³   
1                       Hanoi  Vietnam  Asia/Bangkok      21632  pm25 µg/m³   
2                  nồng độ pm  Vietnam  Asia/Bangkok    1742521  pm25 µg/m³   
3              Cung thiếu nhi  Vietnam  Asia/Bangkok    7772083    co µg/m³   
4              Cung thiếu nhi  Vietnam  Asia/Bangkok    7771978   no2 µg/m³   
5              Cung thiếu nhi  Vietnam  Asia/Bangkok    7772082  pm10 µg/m³   
6              Cung thiếu nhi  Vietnam  Asia/Bangkok    7772104  pm25 µg/m³   
7                    Đầm Trấu  Vietnam  Asia/Bangkok    7772081    co µg/m³   
8                    Đầm Trấu  Vietnam  Asia/Bangkok    7772045   no2 µg/m³   
9                    Đầm Trấu  Vietnam  Asia/Bangkok    7771991  pm10 µg/m³   
10                   Đầm Trấu  Vietnam  Asia/Bangkok    7771992  pm25 µg/m³   
11                 Đào Duy Từ  Vietnam  Asia/Bangkok

In [46]:
# Lấy dữ liệu đo lường từ API sensors
measurements_dict = {}

data = 'measurements'
rollup = 'hourly'
datetime_from = '2025-01-01'
datetime_to = '2025-02-01'

In [47]:
for sensor in sensor_radius_list:
    measurements_list_data = []

    measurements_response = client.measurements.list(
        sensors_id=sensor,
        data=data,
        rollup=rollup,
        datetime_from=datetime_from,
        datetime_to=datetime_to
    )

    if measurements_response.results:
        for result in measurements_response.results:
            date_utc = datetime.strptime(result.period.datetime_from.utc, "%Y-%m-%dT%H:%M:%SZ")
            formatted_date = date_utc.strftime("%Y-%m-%d %H:%M:%S")

            measurements_list_data.append({
                "Date UTC": formatted_date,
                "Sensor ID": sensor,
                "Parameter": result.parameter.name,
                "Unit": result.parameter.units,
                "Value": result.value
            })

        # Chuyển danh sách thành DataFrame
        df = pd.DataFrame(measurements_list_data)
        df["Date UTC"] = pd.to_datetime(df["Date UTC"])

        # Lưu vào dictionary
        measurements_dict[f"sensor_{sensor}_data"] = df

        print(f"Sensor ID: {sensor}, Fetched shape: {df.shape}")

print(f"Fetch Success")

Sensor ID: 21632, Fetched shape: (744, 5)
Sensor ID: 7772083, Fetched shape: (722, 5)
Sensor ID: 7771978, Fetched shape: (722, 5)
Sensor ID: 7772082, Fetched shape: (722, 5)
Sensor ID: 7772104, Fetched shape: (722, 5)
Sensor ID: 7771983, Fetched shape: (723, 5)
Sensor ID: 7772030, Fetched shape: (723, 5)
Sensor ID: 7772101, Fetched shape: (723, 5)
Sensor ID: 7772040, Fetched shape: (723, 5)
Fetch Success


In [57]:
for sensor, df in measurements_dict.items():
    df.to_csv(f"data/{sensor}.csv", index=False, encoding="utf-8")


In [60]:
# Gộp tất cả dữ liệu từ các sensor vào một DataFrame chung
merged_df = pd.concat(measurements_dict.values(), ignore_index=True)

# Gộp cột Parameter và Unit thành một cột duy nhất "Parameter (Unit)"
merged_df["Parameter (Unit)"] = merged_df["Parameter"] + " (" + merged_df["Unit"] + ")"

# Bỏ qua các giá trị âm trước khi tính trung bình
merged_df = merged_df[merged_df["Value"] >= 0]

# Nhóm dữ liệu theo "Date UTC" và "Parameter (Unit)", tính trung bình
df_grouped = merged_df.groupby(["Date UTC", "Parameter (Unit)"], as_index=False)["Value"].mean()

# Tạo dictionary chứa các bảng riêng cho từng Parameter
parameter_tables = {}

# Duyệt qua từng Parameter để tạo bảng riêng
for param in df_grouped["Parameter (Unit)"].unique():
    df_param = df_grouped[df_grouped["Parameter (Unit)"] == param][["Date UTC", "Value"]].copy()
    df_param.rename(columns={"Value": param}, inplace=True)
    parameter_tables[param] = df_param

    print(f"Created table for {param}, shape: {df_param.shape}")

# Kiểm tra một bảng cụ thể (ví dụ: pm25 (µg/m³))
print(parameter_tables.get("pm25 (µg/m³)", "Parameter not found"))


Created table for co (µg/m³), shape: (723, 2)
Created table for no2 (µg/m³), shape: (723, 2)
Created table for pm10 (µg/m³), shape: (723, 2)
Created table for pm25 (µg/m³), shape: (740, 2)
                Date UTC  pm25 (µg/m³)
3    2024-12-31 17:00:00     85.900000
7    2024-12-31 21:00:00     88.600000
11   2024-12-31 23:00:00    109.400000
15   2025-01-01 00:00:00    112.200000
19   2025-01-01 01:00:00    106.600000
...                  ...           ...
2892 2025-01-31 12:00:00     32.300000
2896 2025-01-31 13:00:00     22.600000
2900 2025-01-31 14:00:00     17.366667
2904 2025-01-31 15:00:00     23.700000
2908 2025-01-31 16:00:00     28.000000

[740 rows x 2 columns]


In [67]:
# Gộp tất cả dữ liệu từ các sensor vào một DataFrame chung
merged_df = pd.concat(measurements_dict.values(), ignore_index=True)

# Gộp cột Parameter và Unit thành một cột duy nhất "Parameter (Unit)"
merged_df["Parameter (Unit)"] = merged_df["Parameter"] + " (" + merged_df["Unit"] + ")"

# Bỏ qua các giá trị âm trước khi tính trung bình
merged_df = merged_df[merged_df["Value"] >= 0]

# Nhóm dữ liệu theo "Date UTC" và "Parameter (Unit)", tính trung bình
df_grouped = merged_df.groupby(["Date UTC", "Parameter (Unit)"], as_index=False)["Value"].mean()

# Pivot để chuyển các Parameter thành cột
df_sensor_data = df_grouped.pivot(index="Date UTC", columns="Parameter (Unit)", values="Value").reset_index()

# Bỏ tên cột index của pivot
df_sensor_data.columns.name = None

# Xử lý dữ liệu bị thiếu (Chọn 1 trong 2 cách dưới đây nếu cần)
# df_final.fillna(0, inplace=True)  # Điền 0 nếu không có dữ liệu (nếu cần)
# df_final.fillna(method='ffill', inplace=True)  # Điền giá trị gần nhất trước đó

# Hiển thị bảng hợp nhất
print(df_sensor_data.head())

             Date UTC  co (µg/m³)  no2 (µg/m³)  pm10 (µg/m³)  pm25 (µg/m³)
0 2024-12-31 17:00:00      1935.0        75.20         124.0          85.9
1 2024-12-31 21:00:00      1520.0        58.60         128.0          88.6
2 2024-12-31 23:00:00      1635.0        56.45         158.5         109.4
3 2025-01-01 00:00:00      1560.0        59.95         162.5         112.2
4 2025-01-01 01:00:00      1295.0        62.45         147.5         106.6


In [65]:
# Lấy dữ liệu thời tiết từ Meteostat
from meteostat import Hourly, Point

hanoi = Point(21.0285, 105.8542)

start = pd.Timestamp(datetime(2025, 1, 1))
end = pd.Timestamp(datetime(2025, 2, 1))

df_weather_data = Hourly(hanoi, start, end).fetch()

# Đồng bộ cột Index để merge
df_weather_data = df_weather_data.reset_index()
df_weather_data.rename(columns={"time": "Date UTC"}, inplace=True)

# In kết quả
print(df_weather_data.head())

             Date UTC  temp  dwpt  rhum  prcp  snow   wdir  wspd  wpgt  \
0 2025-01-01 00:00:00  12.0  11.1  94.0   0.0   NaN    0.0   0.0   NaN   
1 2025-01-01 01:00:00  14.0  13.0  94.0   0.0   NaN  100.0   5.4   NaN   
2 2025-01-01 02:00:00  17.0  12.9  77.0   0.0   NaN   80.0   5.4   NaN   
3 2025-01-01 03:00:00  19.0  12.0  64.0   0.0   NaN   80.0   7.6   NaN   
4 2025-01-01 04:00:00  21.0  12.9  60.0   0.0   NaN   90.0   7.6   NaN   

     pres  tsun  coco  
0  1019.0   NaN   5.0  
1  1020.0   NaN   5.0  
2  1020.0   NaN   5.0  
3  1020.0   NaN   5.0  
4  1019.0   NaN   5.0  


In [69]:
# Merge 2 table theo cột Index: "Date UTC"
df_final = pd.merge(df_sensor_data, df_weather_data, on="Date UTC", how="outer")

In [72]:
# Xuất kết quả ra file CSV
df_final.to_csv("data/crawl.csv", index=False, encoding="utf-8")